In [184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [185]:
# Dropping the columns that will not add any value to the model output. While these columns might be useful 

pd.set_option('display.max_columns', None)
df = pd.read_csv(r'C:\Users\Vipul\CreditLine-Decision-Engine-Using-Amazon-Sagemaker\credit.csv')
df.drop(['issue_d','purpose','title','earliest_cr_line','initial_list_status','revol_bal','installment','int_rate','application_type','total_acc','emp_title'],axis=1,inplace = True)


In [186]:
df.head(5)

,loan_amnt,term,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,dti,open_acc,pub_rec,revol_util,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,B,B4,10+ years,RENT,117000.0,Not Verified,Fully Paid,26.24,16.0,0.0,41.8,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,B,B5,4 years,MORTGAGE,65000.0,Not Verified,Fully Paid,22.05,17.0,0.0,53.3,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,B,B3,< 1 year,RENT,43057.0,Source Verified,Fully Paid,12.79,13.0,0.0,92.2,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,A,A2,6 years,RENT,54000.0,Not Verified,Fully Paid,2.60,6.0,0.0,21.5,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,C,C5,9 years,MORTGAGE,55000.0,Verified,Charged Off,33.95,13.0,0.0,69.8,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"


In [187]:
df['pincode'] = df['address'].str.extract(r'(\d{5})$')
df.drop('address',axis=1,inplace=True)

In [188]:
df.head(1)

,loan_amnt,term,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,dti,open_acc,pub_rec,revol_util,mort_acc,pub_rec_bankruptcies,pincode
0,10000.0,36 months,B,B4,10+ years,RENT,117000.0,Not Verified,Fully Paid,26.24,16.0,0.0,41.8,0.0,0.0,22690


In [189]:
df.value_counts('home_ownership')

home_ownership
MORTGAGE    198348
RENT        159790
OWN          37746
OTHER          112
NONE            31
ANY              3
Name: count, dtype: int64

In [190]:
df.value_counts('emp_length')

emp_length
10+ years    126041
2 years       35827
< 1 year      31725
3 years       31665
5 years       26495
1 year        25882
4 years       23952
6 years       20841
7 years       20819
8 years       19168
9 years       15314
Name: count, dtype: int64

In [191]:
df.value_counts('pincode')

pincode
70466    56985
30723    56546
22690    56527
48052    55917
00813    45824
29597    45471
05113    45402
11650    11226
93700    11151
86630    10981
Name: count, dtype: int64

In [192]:
df.value_counts('pub_rec')

pub_rec
0.0     338272
1.0      49739
2.0       5476
3.0       1521
4.0        527
5.0        237
6.0        122
7.0         56
8.0         34
9.0         12
10.0        11
11.0         8
12.0         4
13.0         4
19.0         2
15.0         1
17.0         1
24.0         1
40.0         1
86.0         1
Name: count, dtype: int64

In [193]:

df.value_counts('mort_acc')

mort_acc
0.0     139777
1.0      60416
2.0      49948
3.0      38049
4.0      27887
5.0      18194
6.0      11069
7.0       6052
8.0       3121
9.0       1656
10.0       865
11.0       479
12.0       264
13.0       146
14.0       107
15.0        61
16.0        37
17.0        22
18.0        18
19.0        15
20.0        13
24.0        10
22.0         7
21.0         4
25.0         4
27.0         3
23.0         2
26.0         2
31.0         2
32.0         2
28.0         1
30.0         1
34.0         1
Name: count, dtype: int64

In [194]:
df.value_counts('pub_rec_bankruptcies')


pub_rec_bankruptcies
0.0    350380
1.0     42790
2.0      1847
3.0       351
4.0        82
5.0        32
6.0         7
7.0         4
8.0         2
Name: count, dtype: int64

Dropping this 'total_acc' column as well as it is not adding any additional value to the model. There are some discrepancies as well. Someone with less than 1 year of experience is having 26 accounts, which is not possible.

Even if someone has completed 1 loan from Loan Tap successfully, he should be eligible for another loan.

In [195]:
df.shape 

(396030, 16)

In [196]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             396030 non-null  float64
 1   term                  396030 non-null  object 
 2   grade                 396030 non-null  object 
 3   sub_grade             396030 non-null  object 
 4   emp_length            377729 non-null  object 
 5   home_ownership        396030 non-null  object 
 6   annual_inc            396030 non-null  float64
 7   verification_status   396030 non-null  object 
 8   loan_status           396030 non-null  object 
 9   dti                   396030 non-null  float64
 10  open_acc              396030 non-null  float64
 11  pub_rec               396030 non-null  float64
 12  revol_util            395754 non-null  float64
 13  mort_acc              358235 non-null  float64
 14  pub_rec_bankruptcies  395495 non-null  float64
 15  

In [197]:
df.isnull().sum()

loan_amnt                   0
term                        0
grade                       0
sub_grade                   0
emp_length              18301
home_ownership              0
annual_inc                  0
verification_status         0
loan_status                 0
dti                         0
open_acc                    0
pub_rec                     0
revol_util                276
mort_acc                37795
pub_rec_bankruptcies      535
pincode                     0
dtype: int64

In [198]:
categorical_column = []
nemrical_column = []

for i in df.columns:
    if df[i].dtype == 'object':
        categorical_column.append(i)
    else:
        nemrical_column.append(i)

In [199]:
# Number of unique values in all non-numeric columns
for col in categorical_column:
  print(f"No. of unique values in {col}: {df[col].nunique()}")

No. of unique values in term: 2
No. of unique values in grade: 7
No. of unique values in sub_grade: 35
No. of unique values in emp_length: 11
No. of unique values in home_ownership: 6
No. of unique values in verification_status: 3
No. of unique values in loan_status: 2
No. of unique values in pincode: 10


## Data Engineering

In [200]:
df.head(3)

,loan_amnt,term,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,dti,open_acc,pub_rec,revol_util,mort_acc,pub_rec_bankruptcies,pincode
0,10000.0,36 months,B,B4,10+ years,RENT,117000.0,Not Verified,Fully Paid,26.24,16.0,0.0,41.8,0.0,0.0,22690
1,8000.0,36 months,B,B5,4 years,MORTGAGE,65000.0,Not Verified,Fully Paid,22.05,17.0,0.0,53.3,3.0,0.0,05113
2,15600.0,36 months,B,B3,< 1 year,RENT,43057.0,Source Verified,Fully Paid,12.79,13.0,0.0,92.2,0.0,0.0,05113


### Data Engineering on Categorical columns

In [201]:
#converting months into years.
df['term'] = df['term'].astype(str)
df['term'] = df['term'].str.strip().map({'36 months': 3, '60 months': 5}).astype(int)



In [202]:
#dropping emp_length column as it doesnt make any sense. Now all the employees are eligible. No one is having less than 1 year of experience.
df.drop('emp_length', axis=1, inplace=True)

# DROPPING values that contain home_ownership as OTHER', 'NONE', 'ANY. We cannot risk giving loan to someone we dont know where they are living.
df = df[~df['home_ownership'].isin(['OTHER', 'NONE', 'ANY'])]

In [203]:
df.head(3)

,loan_amnt,term,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,dti,open_acc,pub_rec,revol_util,mort_acc,pub_rec_bankruptcies,pincode
0,10000.0,3,B,B4,RENT,117000.0,Not Verified,Fully Paid,26.24,16.0,0.0,41.8,0.0,0.0,22690
1,8000.0,3,B,B5,MORTGAGE,65000.0,Not Verified,Fully Paid,22.05,17.0,0.0,53.3,3.0,0.0,05113
2,15600.0,3,B,B3,RENT,43057.0,Source Verified,Fully Paid,12.79,13.0,0.0,92.2,0.0,0.0,05113


### Working on missing values

In [180]:
missing_values = df.isna().sum()
print(missing_values)

loan_amnt                   0
term                        0
grade                       0
sub_grade                   0
home_ownership              0
annual_inc                  0
verification_status         0
loan_status                 0
dti                         0
open_acc                    0
pub_rec                     0
revol_util                274
mort_acc                37715
pub_rec_bankruptcies      533
pincode                     0
dtype: int64


In [204]:
# Fill 'revol_util' column with average. Since we can assume that they are having average used revolving credit. It is conservative.
df['revol_util'].fillna(df['revol_util'].mean(), inplace=True)

# Fill 'mort_acc' column with 0.0. 
# Ones with having no value for mortgage, we will consider as 0. We will try to be as conservative as possible.
df['mort_acc'].fillna(0.0, inplace=True)

# Drop rows with missing values in 'pub_rec_bankruptcies' column.
# we cannot take any chances since we dont have data for bankruptcies. we will simply drop these rows.
df.dropna(subset=['pub_rec_bankruptcies'], inplace=True)

In [205]:
df.isna().sum().sum()

0

We have worked on all the missing values.

### Working on numerical columns

In [206]:
df.head(3)

,loan_amnt,term,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,dti,open_acc,pub_rec,revol_util,mort_acc,pub_rec_bankruptcies,pincode
0,10000.0,3,B,B4,RENT,117000.0,Not Verified,Fully Paid,26.24,16.0,0.0,41.8,0.0,0.0,22690
1,8000.0,3,B,B5,MORTGAGE,65000.0,Not Verified,Fully Paid,22.05,17.0,0.0,53.3,3.0,0.0,05113
2,15600.0,3,B,B3,RENT,43057.0,Source Verified,Fully Paid,12.79,13.0,0.0,92.2,0.0,0.0,05113
